# Biopython - Esercizio

[MAFFT](https://www.ebi.ac.uk/Tools/msa/mafft/) è un tool di allineamento multiplo sviluppato da EMBL-EBI (European Bioinformatics Institute - European Molecular Biology Laboratory) per sequenze di DNA.

Usare MAFFT (scegliendo ClustalW come formato di output) per allineare i 14 genomi di SARS-CoV-2 presenti nel file `covid-sequences.fasta` sequenziati nel novembre 2021 e scaricati dal sito di [NCBI](https://www.ncbi.nlm.nih.gov/sars-cov-2/). Il primo, con identificatore `NC_045512.2`, è il genoma di riferimento.

Trovare in seguito tutte le variazioni rispetto ai genomi di riferimento.

---

**Variazione**: una colonna nell'allineamento multiplo in cui esiste almeno una sequenza che ha *mismatch* con il riferimento.

Esempio di allineamento multiplo tra tre genomi `G_REF` (*reference*) `G1` e `G2`, che ha quattro variazioni nelle colonne 5, 8, 13 e 16:

    G_REF   AAGCTGATTGCACGC-T
    G1      --GCAGAGTGCAGGCCT
    G2      --GCCGAGTGCACGCCT

**Variazione 5**: `T` nel reference e `A` in G1 e `C` in G2.

**Variazione 8**: `T` nel reference e `G` sia in G1 e G2.

**Variazione 13**: `C` nel reference e `G` in G1.

**Variazione 16**: `-` (cancellazione) nel reference e `C` sia in G1 che in G2.

---

Si richiede di:
- costruire il *data frame* delle variazioni in cui le colonne del *data frame* le colonne di variazione nell'allineamento multiplo e le righe sono indicizzate con l'identificatore del genoma. Non considerare le colonne che cadono nei gap iniziali e finali.
- estrarre il genoma con più variazioni e quello con meno variazioni (rispetto al *reference*)
- ottenere il *data frame* delle variazioni "complete", cioè in cui tutti i genomi variano rispetto al *reference*.
- produrre il *data frame* delle variazioni "stabili", cioé in cui tutti i genomi variano allo stesso modo rispetto al riferimento (hanno la stessa base). 
- ottenere la lista delle colonne in cui c'è un gap nel genoma di riferimento.
- ottenere la lista delle colonne in cui c'è un gap in almeno uno dei genomi (che non siano il *reference*)

Importare Biopython.

Importare il package `AlignIO` che è il package per manipolare file contenenti allineamenti multipli in diversi formati (tra cui `clustal` che è quello del file di input).

### 1) Leggere l'allineamento in input

Il package `AlignIO` mette a disposizione la funzione `read` per leggere un file contenente un allineamento:

       AligIO.read(input_file_name, format)
       
e restituisce un oggetto `MultipleSeqAlignment` che è un oggetto iterabile contenente oggetti `SeqRecord`, un oggetto per ognuna delle righe dell'allineamento letto.

Ottenere la lunghezza dell'allineamento letto, intesa come numero di colonne della matrice di allineamento, tramite il metodo `get_alignment_length()` dell'oggetto `MultipleSeqAlignment`.

Trasformare l'allineamento in una lista di oggetti `SeqRecord`.

### 2) Eliminare dall'allineamento i gap iniziali.

Trovare il più lungo prefisso di soli simboli `-` presente nelle righe dell'allineamento. Supponendo che tale prefisso sia lungo `g`, eliminare da ogni riga dell'allineamento il prefisso di lunghezza `g`.

Ad esempio il seguente allineamento composto da tre righe:

    GTATGTGTCATGTTTTTGCTA
    --ATGTGTCATG-TTT-----
    ----GTGTCATGTTTTTG---
    
presenta un più lungo prefisso di soli simboli `-` di lunghezza `g=4` (terza riga). Eliminando da tutte le righe un prefisso di lunghezza 4 si ottiene:

        GTGTCATGTTTTTGCTA
        GTGTCATG-TTT-----
        GTGTCATGTTTTTG---

a) Estrarre la lista dei gap iniziali.

In [53]:
gap_list

[[],
 ['---------------------'],
 ['---------------'],
 ['----------------------------------------------'],
 ['----------------------------------------------'],
 ['----------------------------------------------'],
 ['----------------------------------------------'],
 ['----------------------------------------------'],
 ['----------------------------------------------'],
 ['----------------------------------------------'],
 ['-----------------'],
 ['----------------------------------------------'],
 ['----------------------------------------------'],
 ['-----------------']]

b) Estrarre la lista delle posizioni in cui il gap iniziale è nullo.

In [55]:
null_gap_index

[0]

c) Estrarre la lista dei gap non nulli.

In [57]:
gap_list

['---------------------',
 '---------------',
 '----------------------------------------------',
 '----------------------------------------------',
 '----------------------------------------------',
 '----------------------------------------------',
 '----------------------------------------------',
 '----------------------------------------------',
 '----------------------------------------------',
 '-----------------',
 '----------------------------------------------',
 '----------------------------------------------',
 '-----------------']

d) Aggiungere, alla lista precedente, delle stringhe nulle in corrispondenza dei gap iniziali nulli.

In [59]:
gap_list

['',
 '---------------------',
 '---------------',
 '----------------------------------------------',
 '----------------------------------------------',
 '----------------------------------------------',
 '----------------------------------------------',
 '----------------------------------------------',
 '----------------------------------------------',
 '----------------------------------------------',
 '-----------------',
 '----------------------------------------------',
 '----------------------------------------------',
 '-----------------']

e) Determinare la lista delle lunghezze dei gap.

In [61]:
gap_size_list

[0, 21, 15, 46, 46, 46, 46, 46, 46, 46, 17, 46, 46, 17]

f) Estrarre la massima lunghezza dei gap.

In [63]:
max_leading_gap_length

46

g) Rimuovere dalle righe dell'allineamento un prefisso di lunghezza pari a quella trovata.

In [65]:
alignment

[SeqRecord(seq=Seq('TCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...AAA'), id='NC_045512.2', name='<unknown name>', description='NC_045512.2', dbxrefs=[]),
 SeqRecord(seq=Seq('TCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...AAA'), id='OL700521.1', name='<unknown name>', description='OL700521.1', dbxrefs=[]),
 SeqRecord(seq=Seq('TCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...AAA'), id='OL700526.1', name='<unknown name>', description='OL700526.1', dbxrefs=[]),
 SeqRecord(seq=Seq('TCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...AAA'), id='OL700531.1', name='<unknown name>', description='OL700531.1', dbxrefs=[]),
 SeqRecord(seq=Seq('TCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...---'), id='OL700532.1', name='<unknown name>', description='OL700532.1', dbxrefs=[]),
 SeqRecord(seq=Seq('TCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...---'), id='OL700537.1', name='<unknown name>', description='OL700537.1', dbxrefs=[]),
 SeqRecord(seq=Seq('

### 3) Eliminare dall'allineamento i gap finali.

Trovare il più lungo suffisso di soli simboli `-` presente nelle righe dell'allineamento. Supponendo che tale suffisso sia lungo `g`, eliminare da ogni riga dell'allineamento il suffisso di lunghezza `g`.

Ad esempio il seguente allineamento composto da tre righe:

        GTGTCATGTTTTTGCTA
        GTGTCATG-TTT-----
        GTGTCATGTTTTTG---
        
presenta un più lungo suffisso di soli simboli `-` di lunghezza `g=5` (seconda riga). Eliminando da tutte le righe un suffisso di lunghezza 5 si ottiene:

        GTGTCATGTTTT
        GTGTCATG-TTT
        GTGTCATGTTTT

a) Estrarre la lista dei gap finali.

In [67]:
gap_list

[[],
 [],
 [],
 [],
 ['-------------------------------------------------------------------------------------------'],
 ['--------------------------------------------------------------------------------------------------------'],
 [],
 [],
 [],
 ['-----------------------------------------------------------------------'],
 ['--------------------------------------------------------------'],
 [],
 [],
 ['---------------------------']]

b) Estrarre la lista delle posizioni in cui il gap finale è nullo.

In [69]:
null_gap_index

[0, 1, 2, 3, 6, 7, 8, 11, 12]

c) Estrarre la lista dei gap non nulli.

In [71]:
gap_list

['-------------------------------------------------------------------------------------------',
 '--------------------------------------------------------------------------------------------------------',
 '-----------------------------------------------------------------------',
 '--------------------------------------------------------------',
 '---------------------------']

d) Aggiungere, alla lista precedente, delle stringhe nulle in corrispondenza dei gap finali nulli.

In [73]:
gap_list

['',
 '',
 '',
 '',
 '-------------------------------------------------------------------------------------------',
 '--------------------------------------------------------------------------------------------------------',
 '',
 '',
 '',
 '-----------------------------------------------------------------------',
 '--------------------------------------------------------------',
 '',
 '',
 '---------------------------']

e) Determinare la lista delle lunghezze dei gap.

In [75]:
gap_size_list

[0, 0, 0, 0, 91, 104, 0, 0, 0, 71, 62, 0, 0, 27]

f) Estrarre la massima lunghezza dei gap.

In [77]:
max_trailing_gap_length

104

g) Rimuovere dalle righe dell'allineamento un suffisso di lunghezza pari a quella trovata.

In [79]:
alignment

[SeqRecord(seq=Seq('TCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...AGA'), id='NC_045512.2', name='<unknown name>', description='NC_045512.2', dbxrefs=[]),
 SeqRecord(seq=Seq('TCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...AGA'), id='OL700521.1', name='<unknown name>', description='OL700521.1', dbxrefs=[]),
 SeqRecord(seq=Seq('TCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...AGA'), id='OL700526.1', name='<unknown name>', description='OL700526.1', dbxrefs=[]),
 SeqRecord(seq=Seq('TCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...AGA'), id='OL700531.1', name='<unknown name>', description='OL700531.1', dbxrefs=[]),
 SeqRecord(seq=Seq('TCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...AGA'), id='OL700532.1', name='<unknown name>', description='OL700532.1', dbxrefs=[]),
 SeqRecord(seq=Seq('TCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...AGA'), id='OL700537.1', name='<unknown name>', description='OL700537.1', dbxrefs=[]),
 SeqRecord(seq=Seq('

### 4) Creare il *data frame* delle variazioni

a) Creare il dizionario dei dati

- `key`: posizione (1-based) della colonna di variazione all'interno dell'allineamento

- `value`: lista delle basi coinvolte nella variazione (il primo elemento si riferisce al *reference*). Se un genoma non presenta differenza rispetto al *reference*, deve essere  inserita la stringa vuota.

Non tenere conto di basi ambigue.

Estrarre la riga del *reference* (e rimuoverla dall'allineamento).

In [84]:
reference

SeqRecord(seq=Seq('TCTCTTGTAGATCTGTTCTCTAAACGAACTTTAAAATCTGTGTGGCTGTCACTC...AGA'), id='NC_045512.2', name='<unknown name>', description='NC_045512.2', dbxrefs=[])

In [86]:
df_data

{186: ['C', '', '', '', '', 'T', '', '', '', '', '', '', '', ''],
 210: ['G', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T'],
 241: ['C', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T', 'T'],
 521: ['G', '', '', '', '', '', '', '', '', '', '', '', '-', ''],
 522: ['T', '', '', '', '', '', '', '', '', '', '', '', '-', ''],
 523: ['T', '', '', '', '', '', '', '', '', '', '', '', '-', ''],
 1048: ['G', '', '', 'T', '', '', '', '', '', '', '', '', '', ''],
 1244: ['G', '', '', '', '', '', '', '', '', '', 'A', '', '', ''],
 1371: ['A', '', '', '', '', '', '', '', '', '', '', 'G', '', ''],
 1616: ['C', '', '', '', '', '', '', '', '', '', '', '', 'A', ''],
 1684: ['C', '', '', '', '', '', '', '', '', '', '', '', 'T', 'T'],
 1843: ['G', '', '', '', '', '', '', '', '', '', '', 'T', '', ''],
 1889: ['C', '', '', '', '', '', '', '', '', 'T', '', '', '', ''],
 2462: ['C', '', '', '', 'T', '', '', '', '', '', '', '', '', ''],
 2929: ['A', '', '', '', '', '', '', '', ''

b) Determinare la lista degli identificatori dei genomi da usare come indici (chiavi primarie).

In [96]:
index_list

['NC_045512.2',
 'OL700521.1',
 'OL700526.1',
 'OL700531.1',
 'OL700532.1',
 'OL700537.1',
 'OL700524.1',
 'OL700530.1',
 'OL700538.1',
 'OL700543.1',
 'OL700544.1',
 'OL700541.1',
 'OL700533.1',
 'OL700545.1']

c) Creare il data frame

    df = pd.DataFrame(df_data, index = index_list)

In [98]:
df

,186,210,241,521,522,523,1048,1244,1371,1616,...,29095,29119,29402,29409,29509,29543,29648,29700,29742,29781
NC_045512.2,C,G,C,G,T,T,G,G,A,C,...,C,C,G,C,C,G,G,A,G,G
OL700521.1,,T,T,,,,,,,,...,,,T,,,,,,T,
OL700526.1,,T,T,,,,,,,,...,,,T,,,,,,T,
OL700531.1,,T,T,,,,T,,,,...,,,T,,,,,,T,
OL700532.1,,T,T,,,,,,,,...,,,T,,T,T,,,T,
OL700537.1,T,T,T,,,,,,,,...,,,T,,,,,,T,
OL700524.1,,T,T,,,,,,,,...,,,T,,T,,T,,T,
OL700530.1,,T,T,,,,,,,,...,,,T,,,,,,T,
OL700538.1,,T,T,,,,,,,,...,T,,T,,,,,,T,
OL700543.1,,T,T,,,,,,,,...,,T,T,,,,,,T,T


### 5) Estrarre il genoma con più variazioni e quello con meno variazioni

a) Determinare la lista del numero di variazioni per genoma rispetto al riferimento.

In [101]:
variants_per_genome

[49, 53, 54, 58, 58, 58, 55, 60, 58, 54, 55, 56, 54]

b) Estrarre il genoma con più variazioni.

'OL700538.1'

c) Estrarre il genoma con meno variazioni.

'OL700521.1'

### 6) Estrarre il *data frame* delle variazioni "complete"

In [106]:
df_complete

,210,241,3037,14408,15451,16466,21618,21987,22029,22030,...,28249,28250,28251,28252,28253,28271,28461,28881,29402,29742
NC_045512.2,G,C,C,C,G,C,C,G,A,G,...,A,T,T,T,C,A,A,G,G,G
OL700521.1,T,T,T,T,A,T,G,A,-,-,...,-,-,-,-,-,-,G,T,T,T
OL700526.1,T,T,T,T,A,T,G,A,-,-,...,-,-,-,-,-,-,G,T,T,T
OL700531.1,T,T,T,T,A,T,G,A,-,-,...,-,-,-,-,-,-,G,T,T,T
OL700532.1,T,T,T,T,A,T,G,A,-,-,...,-,-,-,-,-,-,G,T,T,T
OL700537.1,T,T,T,T,A,T,G,A,-,-,...,-,-,-,-,-,-,G,T,T,T
OL700524.1,T,T,T,T,A,T,G,A,-,-,...,-,-,-,-,-,-,G,T,T,T
OL700530.1,T,T,T,T,A,T,G,A,-,-,...,-,-,-,-,-,-,G,T,T,T
OL700538.1,T,T,T,T,A,T,G,A,-,-,...,-,-,-,-,-,-,G,T,T,T
OL700543.1,T,T,T,T,A,T,G,A,-,-,...,-,-,-,-,-,-,G,T,T,T


### 7) Estrarre il *data frame* delle variazioni "stabili"

In [108]:
df_stable

,210,241,3037,14408,15451,16466,21618,21987,22029,22030,...,28249,28250,28251,28252,28253,28271,28461,28881,29402,29742
NC_045512.2,G,C,C,C,G,C,C,G,A,G,...,A,T,T,T,C,A,A,G,G,G
OL700521.1,T,T,T,T,A,T,G,A,-,-,...,-,-,-,-,-,-,G,T,T,T
OL700526.1,T,T,T,T,A,T,G,A,-,-,...,-,-,-,-,-,-,G,T,T,T
OL700531.1,T,T,T,T,A,T,G,A,-,-,...,-,-,-,-,-,-,G,T,T,T
OL700532.1,T,T,T,T,A,T,G,A,-,-,...,-,-,-,-,-,-,G,T,T,T
OL700537.1,T,T,T,T,A,T,G,A,-,-,...,-,-,-,-,-,-,G,T,T,T
OL700524.1,T,T,T,T,A,T,G,A,-,-,...,-,-,-,-,-,-,G,T,T,T
OL700530.1,T,T,T,T,A,T,G,A,-,-,...,-,-,-,-,-,-,G,T,T,T
OL700538.1,T,T,T,T,A,T,G,A,-,-,...,-,-,-,-,-,-,G,T,T,T
OL700543.1,T,T,T,T,A,T,G,A,-,-,...,-,-,-,-,-,-,G,T,T,T


### 8) Estrarre la lista delle colonne in cui c'è un gap nel *reference*.

In [110]:
ref_gaps

[]

### 9) Estrarre la lista delle colonne in cui c'è un gap in almeno uno dei genomi (diversi dal *reference*).

In [112]:
other_gaps

[521,
 522,
 523,
 22029,
 22030,
 22031,
 22032,
 22033,
 22034,
 28248,
 28249,
 28250,
 28251,
 28252,
 28253,
 28271]